In [1]:
import leda

import os
key = os.environ['GMAPS_KEY']

leda.enable_d3()
leda.enable_gmaps(key=key)

<IPython.core.display.Javascript object>

In [2]:
leda.render_css('''


html, body, #map {
  width: 100%;
  height: 500px;
  margin: 0;
  padding: 0;
}
.stations, .stations svg {
  position: absolute;
}
.stations svg {
  width: 100px;
  height: 100px;
  padding-right: 100px;
  font: 10px sans-serif;
}
.stations circle {
  fill: brown;
  stroke: black;
  stroke-width: 2px;
}


''')

import networkx as nx
G = nx.Graph()

G.add_node('1', x=37.76487, y=-122.41948, fixed=True)
G.add_node('2', x=38.76, y=-122.0198, fixed=True)
G.add_node('3', x=37.96, y=-121.4198, fixed=True)

G.add_edge('1', '2')
G.add_edge('1', '3')
G.add_edge('2', '3')

from networkx.drawing.nx_agraph import graphviz_layout

pos = graphviz_layout(G)

from networkx.readwrite.json_graph import node_link_data
import json

graph = json.dumps(node_link_data(G))

leda.render_js(u'''
var graph = {{graph}}
Window.data = graph.nodes
''', graph=graph)

<IPython.core.display.Javascript object>

In [3]:
leda.render_html('''
<div id='map'></div>
''')

leda.render_js(u'''
// Create the Google Map
var map = new google.maps.Map(d3.select("#map").node(), {
  zoom: 8,
  center: new google.maps.LatLng(37.76487, -122.41948),
  mapTypeId: google.maps.MapTypeId.TERRAIN
});
  Window.map = map
  
var data = Window.data

var overlay = new google.maps.OverlayView();

// Add the container when the overlay is added to the map.
overlay.onAdd = function() {
var layer = d3.select(this.getPanes().overlayLayer).append("div")
    .attr("class", "stations");

// Draw each marker as a separate SVG element.
// We could use a single SVG, but what size would it have?
overlay.draw = function() {
  var projection = this.getProjection(),
      padding = 15;
    
      var marker = layer.selectAll("svg")
          .data(data)
          .each(transform) // update existing markers
        .enter().append("svg")
          .each(transform)
          .attr("class", "marker");

      // Add a circle.
      marker.append("circle")
          .attr("r", 10)
          .attr("cx", padding)
          .attr("cy", padding);

      // Add a label.
      marker.append("text")
          .attr("x", padding + 15)
          .attr("y", padding)
          .attr("dy", ".31em")
          .text(function(d) { return d.id; });

      function transform(d) {
        console.log(d)
        d = new google.maps.LatLng(d.x, d.y);
        d = projection.fromLatLngToDivPixel(d);
        return d3.select(this)
            .style("left", (d.x - padding) + "px")
            .style("top", (d.y - padding) + "px");
      }
    };
};
  // Bind our overlay to the map…
  overlay.setMap(map);
''')

<IPython.core.display.Javascript object>